In [1]:
import SimpleITK as sitk
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import shutil
import pydicom
import numpy as np


In [2]:
def get_metadata_maybe(key, sitk_img, default='not_found'):
    try:
        value = sitk_img.GetMetaData(key)
    except Exception as e:
        #logging.debug('key not found: {}, {}'.format(key, e))
        value = default
        pass

    return value

In [3]:
def show_3D(sitk_img):
    img_array = sitk.GetArrayFromImage(sitk_img)
    max_depth = img_array.shape[0]
    fig = plt.figure(figsize=(30,max_depth))
    for i in range(max_depth):
        fig.add_subplot(1,max_depth,i+1)
        plt.imshow(img_array[i,:,:])
        plt.axis('off')

In [4]:
def Image3D_from_Time(sitk_img,time):
    img_array = sitk.GetArrayFromImage(sitk_img)
    sitk_array_3D = img_array[time-1,:,:,:] 
    new_sitk_img = sitk.GetImageFromArray(sitk_array_3D)
    return new_sitk_img

In [5]:
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [6]:
#returns img of only one label
def get_single_label_img(sitk_img, label):
    img_array = sitk.GetArrayFromImage(sitk_img)
    label_array = (img_array == label).astype(int)
    label_array = label_array * 100
    label_img = sitk.GetImageFromArray(label_array)
    label_img.CopyInformation(sitk_img)
    return label_img

In [7]:
#returns resampled img1 by referncing img2
def resample_img(sitk_img1, sitk_img2):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(sitk_img2)
    resampled_img = resampler.Execute(sitk_img1)
    return resampled_img

In [8]:
#returns resampled_img back to normal label
def convert_back_to_label(sitk_img, label):
    img_array = sitk.GetArrayFromImage(sitk_img)
    label_array = (img_array > 25).astype(int)
    label_array = label_array * label
    label_img = sitk.GetImageFromArray(label_array)
    label_img.CopyInformation(sitk_img)
    return label_img

In [9]:
#returns resampled_img back to normal label with percentage
def advanced_convert_back_to_label(sitk_img, label, percentage):
    img_array = sitk.GetArrayFromImage(sitk_img)
    percent_array = img_array.flatten()
    percent_array = percent_array[percent_array > 0]
    try:
        threshold = - np.percentile(percent_array*-1,q=percentage)
    except IndexError:
        print("IndexError")
        threshold = 50
    label_array = (img_array >= threshold).astype(int)
    label_array = label_array * label
    label_img = sitk.GetImageFromArray(label_array)
    label_img.CopyInformation(sitk_img)
    return label_img

In [10]:
#returns all labels added and with priority from label 3 to 1
def add_labels(label_array1,label_array2,label_array3):
    final_array = label_array3
    temp_array = final_array + label_array2
    #if you add the label, there is only a 2 if there was a 0 there before, so now oferwriting
    temp_array = (temp_array == 2).astype(int)
    temp_array = temp_array * 2
    final_array = final_array + temp_array
    temp_array = final_array + label_array1
    temp_array = (temp_array == 1).astype(int)
    temp_array = temp_array * 1
    final_array = final_array + temp_array
    return final_array

In [11]:
def add_labels_max_thres(label_array1, label_array2, label_array3, threshold):
    max_label_1_2_array = np.maximum(label_array1, label_array2)
    max_label_2_3_array = np.maximum(label_array2, label_array3)
    max_label_array = np.maximum(max_label_1_2_array, max_label_2_3_array)    
    thres_label_array = (max_label_array >= threshold).astype(int)    
    max_label_array = thres_label_array * max_label_array   
    
    max_label_array1 =  label_array1 * np.equal(max_label_array, label_array1).astype(int)
    #max_label_array1 =  (max_label_array1 > threshold).astype(int) * 1
    max_label_array1 =  max_label_array1.astype(bool).astype(int) * 1
    max_label_array2 =  label_array2 * np.equal(max_label_array, label_array2).astype(int)
    #max_label_array2 =  (max_label_array2 > threshold).astype(int) * 2
    max_label_array2 =  max_label_array2.astype(bool).astype(int) * 2
    max_label_array3 =  label_array3 * np.equal(max_label_array, label_array3).astype(int)
    #max_label_array3 =  (max_label_array3 > threshold).astype(int) * 3
    max_label_array3 =  max_label_array3.astype(bool).astype(int) * 3
    
    return add_labels(max_label_array1, max_label_array2, max_label_array3)    

In [12]:
#reutrns resampled img1 by referencing img2
def resample_label_img(sitk_img1, sitk_img2):
    label1_img1 = get_single_label_img(sitk_img1,1)
    label2_img1 = get_single_label_img(sitk_img1,2)
    label3_img1 = get_single_label_img(sitk_img1,3)
    
    resampled_label1 = resample_img(label1_img1, sitk_img2)
    resampled_label2 = resample_img(label2_img1, sitk_img2)
    resampled_label3 = resample_img(label3_img1, sitk_img2)
    resampled_label1 = convert_back_to_label(resampled_label1,1)
    resampled_label2 = convert_back_to_label(resampled_label2,2)
    resampled_label3 = convert_back_to_label(resampled_label3,3)
    
    resampled_array1 = sitk.GetArrayFromImage(resampled_label1)
    resampled_array2 = sitk.GetArrayFromImage(resampled_label2)
    resampled_array3 = sitk.GetArrayFromImage(resampled_label3)
    resampled_array = add_labels(resampled_array1, resampled_array2, resampled_array3)
    resampled_img= sitk.GetImageFromArray(resampled_array)
    resampled_img.CopyInformation(resampled_label1)
    return resampled_img

In [13]:
#reutrns resampled img1 by referencing img2 with percentage
def percentage_resample_label_img(sitk_img1, sitk_img2, percentage):
    label1_img1 = get_single_label_img(sitk_img1,1)
    label2_img1 = get_single_label_img(sitk_img1,2)
    label3_img1 = get_single_label_img(sitk_img1,3)
    
    resampled_label1 = resample_img(label1_img1, sitk_img2)
    resampled_label2 = resample_img(label2_img1, sitk_img2)
    resampled_label3 = resample_img(label3_img1, sitk_img2)
    resampled_label1 = advanced_convert_back_to_label(resampled_label1,1, percentage)
    resampled_label2 = advanced_convert_back_to_label(resampled_label2,2, percentage)
    resampled_label3 = advanced_convert_back_to_label(resampled_label3,3, percentage)
    
    resampled_array1 = sitk.GetArrayFromImage(resampled_label1)
    resampled_array2 = sitk.GetArrayFromImage(resampled_label2)
    resampled_array3 = sitk.GetArrayFromImage(resampled_label3)
    resampled_array = add_labels(resampled_array1, resampled_array2, resampled_array3)
    resampled_img= sitk.GetImageFromArray(resampled_array)
    resampled_img.CopyInformation(resampled_label1)
    return resampled_img

In [14]:
#reutrns resampled img1 by referencing img2 with percentage
def max_thres_resample_label_img(sitk_img1, sitk_img2, threshold):
    label1_img1 = get_single_label_img(sitk_img1,1)
    label2_img1 = get_single_label_img(sitk_img1,2)
    label3_img1 = get_single_label_img(sitk_img1,3)
    
    resampled_label1 = resample_img(label1_img1, sitk_img2)
    resampled_label2 = resample_img(label2_img1, sitk_img2)
    resampled_label3 = resample_img(label3_img1, sitk_img2) 
    
    resampled_array1 = sitk.GetArrayFromImage(resampled_label1)
    resampled_array2 = sitk.GetArrayFromImage(resampled_label2)
    resampled_array3 = sitk.GetArrayFromImage(resampled_label3)
    
    resampled_array = add_labels_max_thres(resampled_array1, resampled_array2, resampled_array3, threshold)
    resampled_img= sitk.GetImageFromArray(resampled_array)
    resampled_img.CopyInformation(resampled_label1)
    return resampled_img

In [15]:
#reutrns resampled img1 by referencing img2 with percentage
def max_thres_resample2_label_img(sitk_img1, sitk_img2, threshold):
    label1_img1 = get_single_label_img(sitk_img1,1)
    label2_img1 = get_single_label_img(sitk_img1,2)
    label3_img1 = get_single_label_img(sitk_img1,3)
    
    resampled_label1 = resample_direcion_origin_spacing(label1_img1, sitk_img2, interpolate=sitk.sitkLinear)
    resampled_label2 = resample_direcion_origin_spacing(label2_img1, sitk_img2, interpolate=sitk.sitkLinear)
    resampled_label3 = resample_direcion_origin_spacing(label3_img1, sitk_img2, interpolate=sitk.sitkLinear) 
    
    resampled_array1 = sitk.GetArrayFromImage(resampled_label1)
    resampled_array2 = sitk.GetArrayFromImage(resampled_label2)
    resampled_array3 = sitk.GetArrayFromImage(resampled_label3)
    
    resampled_array = add_labels_max_thres(resampled_array1, resampled_array2, resampled_array3, threshold)
    resampled_img= sitk.GetImageFromArray(resampled_array)
    resampled_img = sitk.Cast(resampled_img, sitk.sitkUInt8)
    resampled_img.CopyInformation(resampled_label1)
    return resampled_img

In [18]:
def resample_direcion_origin_spacing(sitk_img, reference_sitk, interpolate=sitk.sitkLinear):
    """
    Resample a sitk img, copy direction, origin and spacing of the reference image
    Keep the size (resolution) of the target image
    :param sitk_img: sitk.Image
    :param reference_sitk: sitk.Image
    :param interpolate: fn
    :return: 
    """
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(reference_sitk.GetSpacing())
    resampler.SetInterpolator(interpolate)
    resampler.SetOutputDirection(reference_sitk.GetDirection())
    resampler.SetOutputOrigin(reference_sitk.GetOrigin())
    resampler.SetSize(sitk_img.GetSize())
    resampled = resampler.Execute(sitk_img)
    # copy metadata
    for key in reference_sitk.GetMetaDataKeys():
        resampled.SetMetaData(key, get_metadata_maybe(reference_sitk, key))
    return resampled

In [17]:
#reutrns resampled img1 by referencing img2 with percentage
def max_thres_resample2_iso_label_img(sitk_img1, threshold):
    label1_img1 = get_single_label_img(sitk_img1,1)
    label2_img1 = get_single_label_img(sitk_img1,2)
    label3_img1 = get_single_label_img(sitk_img1,3)
    
    resampled_label1 = transform_to_isotrop_voxels(label1_img1, interpolate=sitk.sitkLinear, spacing_=(1.5,1.5,1.5))
    resampled_label2 = transform_to_isotrop_voxels(label2_img1, interpolate=sitk.sitkLinear, spacing_=(1.5,1.5,1.5))
    resampled_label3 = transform_to_isotrop_voxels(label3_img1, interpolate=sitk.sitkLinear, spacing_=(1.5,1.5,1.5))
    
    resampled_array1 = sitk.GetArrayFromImage(resampled_label1)
    resampled_array2 = sitk.GetArrayFromImage(resampled_label2)
    resampled_array3 = sitk.GetArrayFromImage(resampled_label3)
    
    resampled_array = add_labels_max_thres(resampled_array1, resampled_array2, resampled_array3, threshold)
    resampled_img= sitk.GetImageFromArray(resampled_array)
    resampled_img = sitk.Cast(resampled_img, sitk.sitkUInt8)
    resampled_img.CopyInformation(resampled_label1)
    return resampled_img

In [39]:
def transform_to_isotrop_voxels(sitk_img, interpolate=sitk.sitkBSpline, spacing_=(1.,1.,1.)):

    size = sitk_img.GetSize()
    spacing = sitk_img.GetSpacing()
    size_new = tuple([int((s*space_old)//space_new) for s,space_old,space_new in zip(size,spacing,spacing_)])

    # resample to isotrop voxels
    resampler = sitk.ResampleImageFilter()
    resampler.SetSize(size_new)
    resampler.SetOutputSpacing(spacing_)
    resampler.SetOutputOrigin(sitk_img.GetOrigin())
    resampler.SetOutputDirection(sitk_img.GetDirection())
    resampler.SetInterpolator(interpolate) 
    resampled = resampler.Execute(sitk_img)
        # copy metadata
    for key in sitk_img.GetMetaDataKeys():
        resampled.SetMetaData(key, get_metadata_maybe(sitk_img, key))
    return resampled

In [41]:
#origin shift with transformindex
def resample_direcion_origin_spacing_shift(sitk_img, reference_sitk, shift, interpolate=sitk.sitkLinear):
    """
    Resample a sitk img, copy direction, origin and spacing of the reference image
    Keep the size (resolution) of the target image
    :param sitk_img: sitk.Image
    :param reference_sitk: sitk.Image
    :param interpolate: fn
    :return: falsch muss umgeschrieben werden
    """
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(reference_sitk.GetSpacing())
    resampler.SetInterpolator(interpolate)
    resampler.SetOutputDirection(reference_sitk.GetDirection())
    #if coupled with resampler.SetSize only shift origin if negativ
    resampler.SetOutputOrigin(reference_sitk.TransformIndexToPhysicalPoint(tuple([abs(x)*-1 for x in shift])))
    #resampler.SetSize(sitk_img.GetSize())
    resampler.SetSize(tuple(map(sum,zip(reference_sitk.GetSize(), tuple([abs(x)*2 for x in shift])))))
    #resampler.SetSize(tuple(map(sum,zip(sitk_img.GetSize(), tuple([abs(x) for x in shift])))))
    resampled = resampler.Execute(sitk_img)
    # copy metadata
    for key in reference_sitk.GetMetaDataKeys():
        resampled.SetMetaData(key, get_metadata_maybe(reference_sitk, key))
    return resampled


#reutrns resampled img1 by referencing img2 with percentage
def max_thres_resample2_label_img_shift(sitk_img1, sitk_img2, threshold, shift):
    label1_img1 = get_single_label_img(sitk_img1,1)
    label2_img1 = get_single_label_img(sitk_img1,2)
    label3_img1 = get_single_label_img(sitk_img1,3)
    
    resampled_label1 = resample_direcion_origin_spacing_shift(label1_img1, sitk_img2, shift, interpolate=sitk.sitkLinear)
    resampled_label2 = resample_direcion_origin_spacing_shift(label2_img1, sitk_img2, shift, interpolate=sitk.sitkLinear)
    resampled_label3 = resample_direcion_origin_spacing_shift(label3_img1, sitk_img2, shift, interpolate=sitk.sitkLinear) 
    
    resampled_array1 = sitk.GetArrayFromImage(resampled_label1)
    resampled_array2 = sitk.GetArrayFromImage(resampled_label2)
    resampled_array3 = sitk.GetArrayFromImage(resampled_label3)
    
    resampled_array = add_labels_max_thres(resampled_array1, resampled_array2, resampled_array3, threshold)
    resampled_img= sitk.GetImageFromArray(resampled_array)
    resampled_img = sitk.Cast(resampled_img, sitk.sitkUInt8)
    resampled_img.CopyInformation(resampled_label1)
    return resampled_img

In [22]:
ax_src = '/home/rflo/master/all_with_7_worst_regi/ax3d/'
sax_src = '/home/rflo/master/all_with_7_worst_regi/sax3d/'
dst = '/home/rflo/master/all_with_7_worst_regi/testground/'

In [ ]:
#needs 
#ax_src = path to directory with every axial 3d nrrd file
#sax_src = path to directory with every short axial 3d nrrd file
#dst = path to directory with where ax3d_iso_linear, sax3d_iso_linear, ax_to_sax_iso_linear and ax_to_sax_to_ax_iso_linear get saved


#resample axial 3D Images into isotrop and saves it into dst/ax3d_iso_linear/

ax_iso_path = os.path.join(dst, 'ax3d_iso_linear/')
ensure_dir(ax_iso_path)

for file_path in sorted(glob.glob(os.path.join(ax_src+'*.nrrd'), recursive = True)):

    reader1 = sitk.ImageFileReader()
    reader1.SetFileName(file_path)
    sitk_img = reader1.Execute()
    
    if 'img' in os.path.basename(file_path):
        iso_resampled = transform_to_isotrop_voxels(sitk_img, interpolate=sitk.sitkLinear, spacing_=(1.5,1.5,1.5))
    else:
        iso_resampled = max_thres_resample2_iso_label_img(sitk_img, 50)
    
    writer = sitk.ImageFileWriter()
    writer.SetFileName(os.path.join(ax_iso_path,os.path.basename(file_path)))
    writer.Execute(iso_resampled) 

print('ax3d_iso_linear finished')


#resamples short axes 3D Images into isotrop and saves them into dst/sax3d_iso_linear/

sax_iso_path = os.path.join(dst, 'sax3d_iso_linear/')
ensure_dir(sax_iso_path)

for file_path in sorted(glob.glob(os.path.join(sax_src+'*.nrrd'), recursive = True)):    

    reader1 = sitk.ImageFileReader()
    reader1.SetFileName(file_path)
    sitk_img = reader1.Execute()
    
    if 'img' in os.path.basename(file_path):
        iso_resampled = transform_to_isotrop_voxels(sitk_img, interpolate=sitk.sitkLinear, spacing_=(1.5,1.5,1.5))
    else:
        iso_resampled = max_thres_resample2_iso_label_img(sitk_img, 50)
    
    writer = sitk.ImageFileWriter()
    writer.SetFileName(os.path.join(sax_iso_path,os.path.basename(file_path)))
    writer.Execute(iso_resampled)
    
print('sax3d_iso_linear finished')   


#resamples axial 3D Images into short axes 3D Images and saves them into dst/ax_to_sax_iso_linear/
#then resamples them back into axial 3D Images and saves them into dst/ax_to_sax_to_ax_iso_linear/

ax_to_sax_iso_path = os.path.join(dst, 'ax_to_sax_iso_linear/')
ensure_dir(ax_to_sax_iso_path)
ax_to_sax_to_ax_iso_path = os.path.join(dst, 'ax_to_sax_to_ax_iso_linear/')
ensure_dir(ax_to_sax_to_ax_iso_path)

for file_path in sorted(glob.glob(os.path.join(ax_iso_path+'*.nrrd'), recursive = True)):
    
    ax_file_path = file_path
    sax_file_path = os.path.join(sax_iso_path, os.path.basename(file_path))

    reader1 = sitk.ImageFileReader()
    reader1.SetFileName(ax_file_path)
    ax_img = reader1.Execute()

    reader2 = sitk.ImageFileReader()
    reader2.SetFileName(sax_file_path)
    sax_img = reader2.Execute()
    
    if 'img' in os.path.basename(file_path):
        resampled = resample_direcion_origin_spacing(ax_img, sax_img, interpolate=sitk.sitkLinear)
    else:
        resampled = max_thres_resample2_label_img(ax_img, sax_img, 50)
    
    writer = sitk.ImageFileWriter()
    writer.SetFileName(os.path.join(ax_to_sax_iso_path,os.path.basename(file_path)))
    writer.Execute(resampled)

    if 'img' in os.path.basename(file_path):
        back_resampled = resample_direcion_origin_spacing(resampled, ax_img, interpolate=sitk.sitkLinear)
    else:
        back_resampled = max_thres_resample2_label_img(resampled, ax_img, 50)
    
    writer = sitk.ImageFileWriter()
    writer.SetFileName(os.path.join(ax_to_sax_to_ax_iso_path,os.path.basename(file_path)))
    writer.Execute(back_resampled)

print('ax_to_sax_iso_linear  and ax_to_sax_to_ax_iso_linear finished')

In [20]:
ax_src = '/home/rflo/master/all_with_7_worst_regi/ax3d/'
sax_src = '/home/rflo/master/all_with_7_worst_regi/sax3d/'
dst = '/home/rflo/master/all_with_7_worst_regi/testground/'

In [42]:
#needs 
#ax_src = path to directory with every axial 3d nrrd file
#sax_src = path to directory with every short axial 3d nrrd file
#dst = path to directory with where ax3d_iso_linear, sax3d_iso_linear, ax_to_sax_iso_linear and ax_to_sax_to_ax_iso_linear get saved


#resample axial 3D Images into isotrop and saves it into dst/ax3d_iso_linear/

ax_iso_path = os.path.join(dst, 'ax3d_iso_linear/')
ensure_dir(ax_iso_path)

for file_path in sorted(glob.glob(os.path.join(ax_src+'*.nrrd'), recursive = True)):

    reader1 = sitk.ImageFileReader()
    reader1.SetFileName(file_path)
    sitk_img = reader1.Execute()
    
    if 'img' in os.path.basename(file_path):
        iso_resampled = transform_to_isotrop_voxels(sitk_img, interpolate=sitk.sitkLinear, spacing_=(1.5,1.5,1.5))
    else:
        iso_resampled = max_thres_resample2_iso_label_img(sitk_img, 50)
    
    writer = sitk.ImageFileWriter()
    writer.SetFileName(os.path.join(ax_iso_path,os.path.basename(file_path)))
    writer.Execute(iso_resampled) 

print('ax3d_iso_linear finished')


#resamples short axes 3D Images into isotrop and saves them into dst/sax3d_iso_linear/

sax_iso_path = os.path.join(dst, 'sax3d_iso_linear/')
ensure_dir(sax_iso_path)

for file_path in sorted(glob.glob(os.path.join(sax_src+'*.nrrd'), recursive = True)):    

    reader1 = sitk.ImageFileReader()
    reader1.SetFileName(file_path)
    sitk_img = reader1.Execute()
    
    if 'img' in os.path.basename(file_path):
        iso_resampled = transform_to_isotrop_voxels(sitk_img, interpolate=sitk.sitkLinear, spacing_=(1.5,1.5,1.5))
    else:
        iso_resampled = max_thres_resample2_iso_label_img(sitk_img, 50)
    
    writer = sitk.ImageFileWriter()
    writer.SetFileName(os.path.join(sax_iso_path,os.path.basename(file_path)))
    writer.Execute(iso_resampled)
    
print('sax3d_iso_linear finished')   


#resamples axial 3D Images into short axes 3D Images and saves them into dst/ax_to_sax_iso_linear/
#then resamples them back into axial 3D Images and saves them into dst/ax_to_sax_to_ax_iso_linear/

ax_to_sax_iso_path = os.path.join(dst, 'ax_to_sax_iso_linear/')
ensure_dir(ax_to_sax_iso_path)
ax_to_sax_to_ax_iso_path = os.path.join(dst, 'ax_to_sax_to_ax_iso_linear/')
ensure_dir(ax_to_sax_to_ax_iso_path)

for file_path in sorted(glob.glob(os.path.join(ax_iso_path+'*.nrrd'), recursive = True)):
    
    ax_file_path = file_path
    sax_file_path = os.path.join(sax_iso_path, os.path.basename(file_path))

    reader1 = sitk.ImageFileReader()
    reader1.SetFileName(ax_file_path)
    ax_img = reader1.Execute()

    reader2 = sitk.ImageFileReader()
    reader2.SetFileName(sax_file_path)
    sax_img = reader2.Execute()
    
    if 'img' in os.path.basename(file_path):
        resampled = resample_direcion_origin_spacing_shift(ax_img, sax_img, (0,0,-10), interpolate=sitk.sitkLinear)
    else:
        resampled =resampled = max_thres_resample2_label_img_shift(ax_img, sax_img, 50, (0,0,-10))
    
    writer = sitk.ImageFileWriter()
    writer.SetFileName(os.path.join(ax_to_sax_iso_path,os.path.basename(file_path)))
    writer.Execute(resampled)

    if 'img' in os.path.basename(file_path):
        back_resampled = resample_direcion_origin_spacing(resampled, ax_img, interpolate=sitk.sitkLinear)
    else:
        back_resampled = max_thres_resample2_label_img(resampled, ax_img, 50)
    
    writer = sitk.ImageFileWriter()
    writer.SetFileName(os.path.join(ax_to_sax_to_ax_iso_path,os.path.basename(file_path)))
    writer.Execute(back_resampled)

print('ax_to_sax_iso_linear  and ax_to_sax_to_ax_iso_linear finished')

ax3d_iso_linear finished
sax3d_iso_linear finished
ax_to_sax_iso_linear  and ax_to_sax_to_ax_iso_linear finished
